In [ ]:
import pandas as pb

data = pb.read_excel('FAKSexport.xlsx')
#print(data)
print(data.shape)



In [ ]:
import pandas as pb

Smaldata = pb.read_excel('SmalFAKS.xlsx')
#print(data)
print(Smaldata.shape)


In [ ]:
import pandas as pd

# Konvertera alla kolumnnamn till strängar för att undvika TypeError
Smaldata.columns = Smaldata.columns.astype(str)
data.columns = data.columns.astype(str)

print("1.6")
# Sök efter kolumner som innehåller det specifika ordet
word_to_remove = 'checkbox'
columns_to_remove = [col for col in Smaldata.columns if word_to_remove in col]

print(Smaldata.shape)
print(data.shape)
print("2")
# Ta bort de hittade kolumnerna från DataFrame
data = data.drop(columns=columns_to_remove)

print(Smaldata.shape)
print(data.shape)
print("3")
# Spara den uppdaterade DataFrame till en ny Excel-fil eller samma fil
data.to_excel('updFAKSexp.xlsx', index=False)

In [1]:
import pandas as pb
data = pb.read_excel('updFAKSexp.xlsx')
print(1)
#data.to_csv('updFAKSexpCSV.csv')
data_np = data.to_numpy()
print(data.shape)

1
(129730, 487)


In [ ]:
orgdata = data

In [ ]:
print(data.shape)

In [2]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import numpy as np

model_name = "AI-Sweden-Models/gpt-sw3-1.3b"

#data = pb.read_excel('updFAKSexp.xlsx')
print("1.6")


data = data.astype(str)

#print(data.iloc[:, 0])
#print(data.iloc[:, 1:])

# Tokenisera texten med en tokeniserare
tokenizer = AutoTokenizer.from_pretrained(model_name)


input_tokens = [tokenizer.encode(input_text) for input_text in data.iloc[:, 1:].iterrows()]
print(len(input_tokens))
output_tokens = [tokenizer.encode(output_text) for output_text in data.iloc[:, 0]]



par_tokens = [(output_tokens[i], input_tokens[i])for i in range(len(input_tokens))]

print(par_tokens[3])
print(len(par_tokens))

print("1.6")

# Dela upp datamängden enligt 80-10-10-split
train_texts, val_and_test_texts = train_test_split(par_tokens, test_size=0.2, random_state=42) #måste läsa lite om random_state
val_texts, test_texts = train_test_split(val_and_test_texts, test_size=0.5, random_state=42)
print("1.6")

'''PatientJournalText_train, PatientJournalText_val_and_test = train_test_split(output_tokens, test_size=0.2, random_state=42)
PatientJournalText_val, PatientJournalText_test = train_test_split(PatientJournalText_val_and_test, test_size=0.5, random_state=42)
InputData_train, InputData_val_and_test = train_test_split(input_tokens, test_size=0.2, random_state=42)
InputData_val, InputData_test = train_test_split(InputData_val_and_test, test_size=0.5, random_state=42)         blir random och har då inte rätt par''' 


#print(test_texts)
#print(train_texts)

# Spara de delade datamängderna till separata filer
with open('train_data.txt', 'w') as f:
    for text in train_texts:
        f.write(' '.join(map(str, text)) + '\n')

with open('val_data.txt', 'w') as f:
    for text in val_texts:
        f.write(' '.join(map(str, text)) + '\n')


with open('test_data.txt', 'w') as f:
    for text in test_texts:
        f.write(' '.join(map(str, text)) + '\n')


1.6
129730
([30890, 1160, 8718, 294, 23445, 350, 2572, 15766, 63450, 785, 285, 32439, 401, 1536, 1027, 63446, 4472, 670, 5507, 483, 6579, 383, 5057, 63423, 63484, 63484, 63453, 63497, 63455, 48109, 30890, 279, 434, 8718, 4732, 437, 63443, 30890, 29555, 1738, 425, 324, 1290, 7712, 1414, 63443, 30890, 670, 5507, 483, 10631, 63437, 9657, 14417, 63443, 9435, 25867, 552, 5992, 623, 362, 8390, 63443], (3, A_duration                                 nan
A_duration__REF                      100 - 176
A_mitr_Vmax                                0.7
A_mitr_Vmax__REF                   0,26 - 0,90
age                                         66
                                   ...        
VK_volym_3D_syst_BSA__REF               9 - 29
VK_volym_tei_diast           118.2432432432432
VK_volym_tei_syst                          nan
VKs                                        nan
VKs_BSA                                    nan
Name: 3, Length: 486, dtype: object))
129730
1.6
1.6


In [4]:
output_tokens_from_par_train = [item[0] for item in train_texts]
output_tokens_from_par_val = [item[0] for item in val_texts]
output_tokens_from_par_test = [item[0] for item in test_texts]
input_tokens_from_par_train = [item[1:] for item in train_texts]
input_tokens_from_par_val = [item[1:] for item in val_texts]
input_tokens_from_par_test = [item[1:] for item in test_texts]

PatientJournalText_train_small = output_tokens_from_par_train[:80]
PatientJournalText_val_small = output_tokens_from_par_val[:10]
PatientJournalText_test_small = output_tokens_from_par_test[:10]
InputData_train_small = input_tokens_from_par_train[:80]
InputData_val_small = input_tokens_from_par_val[:10]
InputData_test_small = input_tokens_from_par_test[:10]

In [1]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForCausalLM
import numpy as np
import evaluate

model_name = "AI-Sweden-Models/gpt-sw3-1.3b"

model = AutoModelForCausalLM.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    logging_steps=100,
    save_steps=200,
    logging_dir="./logs")
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset= PatientJournalText_test_small,
    eval_dataset= PatientJournalText_val_small,
    compute_metrics= compute_metrics,
)

trainer.train()


NameError: name 'model_name' is not defined

In [3]:
print("----------------")
output_tokens_from_par = [item[0] for item in par_tokens]
print(output_tokens_from_par[0])
output_tokens_from_par = [item[1:] for item in par_tokens]
print(output_tokens_from_par[0])

----------------
[18870, 12608, 1212, 3750, 332, 14851, 18050, 30503, 63443, 27216, 63453, 3316, 18050, 63423, 63459, 63496, 63489, 63484, 353, 8961, 63446, 63423, 63459, 63443, 63491, 298, 63481, 63435, 63443, 16398, 63481, 823, 602, 34138, 19420, 63423, 63496, 3532, 63470, 63459, 63483, 63446, 467, 63459, 16042, 63423, 63489, 63455, 14959, 18, 59107, 12465, 415, 25867, 552, 533, 459, 662, 305, 900, 2954, 279, 10470, 437, 293, 13014, 63443, 30890, 7675, 8718, 63437, 23445, 350, 6136, 12479, 459, 2481, 63429, 1221, 14792, 30374, 54145, 63446, 13085, 636, 24030, 670, 5507, 483, 6579, 63446, 46939, 5057, 63423, 63480, 63455, 63453, 63480, 63484, 14502, 4116, 5862, 15964, 1090, 338, 51032, 6197, 63423, 63459, 63455, 11679, 1535, 63423, 63456, 63446, 63459, 63491, 416, 63481, 958, 383, 20438, 5196, 2952, 2379, 16413, 21771, 338, 1361, 731, 30890, 7675, 10631, 63437, 23445, 350, 2572, 24030, 670, 5507, 483, 6579, 63443, 21212, 63453, 11415, 2912, 3664, 342, 55218, 63446, 477, 4228, 27473, 4

In [9]:
from transformers.utils.import_utils import _is_package_available
_accelerate_available, _accelerate_version = _is_package_available("accelerate", return_version=True)
print(_accelerate_available)
print(_accelerate_version)

True
0.28.0
